In [1]:
import requests
import json
import re
from tqdm import tqdm
import os
import pandas as pd
import time
import uuid
from jsonschema import validate

In [2]:
def fetch_save_read(url, file, reader=pd.read_csv, sep='\t', **kwargs):
  ''' Download file from {url}, save it to {file}, and subsequently read it with {reader} using pandas options on {**kwargs}.
  '''
  if not os.path.exists(file):
    if os.path.dirname(file):
      os.makedirs(os.path.dirname(file), exist_ok=True)
    df = reader(url, sep=sep, index_col=None)
    df.to_csv(file, sep=sep, index=False)
  return pd.read_csv(file, sep=sep, **kwargs)

In [3]:
organism = "Mammalia/Homo_sapiens"
url = 'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/{}.gene_info.gz'.format(organism)
file = '{}.gene_info.tsv'.format(organism)

ncbi_gene = fetch_save_read(url, file)


In [8]:
ncbi_gene.type_of_gene.unique()

array(['protein-coding', 'pseudo', 'other', 'unknown', 'ncRNA', 'tRNA',
       'rRNA', 'scRNA', 'snoRNA', 'snRNA', 'biological-region'],
      dtype=object)

In [5]:
def maybe_split(record):
    ''' NCBI Stores Nulls as '-' and lists '|' delimited
    '''
    if record in {'', '-'}:
        return set()
    return set(record.split('|'))

def supplement_dbXref_prefix_omitted(ids):
    ''' NCBI Stores external IDS with Foreign:ID while most datasets just use the ID
    '''
    for id in ids:
        # add original id
        yield id
        # also add id *without* prefix
        if ':' in id:
            yield id.split(':', maxsplit=1)[1]

In [6]:
ncbi_gene['All_synonyms'] = [
    set.union(
      maybe_split(gene_info['Symbol']),
      maybe_split(gene_info['Symbol_from_nomenclature_authority']),
      maybe_split(str(gene_info['GeneID'])),
      maybe_split(gene_info['Synonyms']),
      maybe_split(gene_info['Other_designations']),
      maybe_split(gene_info['LocusTag']),
      set(supplement_dbXref_prefix_omitted(maybe_split(gene_info['dbXrefs']))),
    )
    for _, gene_info in ncbi_gene.iterrows()
  ]

synonyms, gene_id = zip(*{
    (synonym, gene_info['GeneID'])
    for _, gene_info in ncbi_gene.iterrows()
    for synonym in gene_info['All_synonyms']
  })
ncbi_lookup_syn = pd.Series(gene_id, index=synonyms)
symbols, cap, gene_id = zip(*{
    (gene_info['Symbol'], gene_info['Symbol'].upper(), gene_info['GeneID'])
    for _, gene_info in ncbi_gene.iterrows()
  })
ncbi_lookup_sym = pd.Series(gene_id, index=symbols)
ncbi_lookup_sym_cap = pd.Series(gene_id, index=cap)

In [9]:
index_values = ncbi_lookup_syn.index.value_counts()
ambiguous = index_values[index_values > 1].index
ncbi_lookup_syn_disambiguated = ncbi_lookup_syn[(
(ncbi_lookup_syn.index == ncbi_lookup_syn) | (~ncbi_lookup_syn.index.isin(ambiguous))
)]
sym_dict = ncbi_lookup_sym.to_dict()
syn_dict_cap = ncbi_lookup_sym_cap.to_dict()
syn_dict = ncbi_lookup_syn_disambiguated.to_dict()
def gene_lookup(gene):
    gene_id = sym_dict.get(gene)
    if gene_id: return str(gene_id)
    gene_id = syn_dict_cap.get(gene)
    if gene_id: return str(gene_id)
    return str(syn_dict.get(gene))

In [10]:
gene_lookup('HLA-A')

'3105'

In [43]:
gene_name_mapper = {
    "KI67": "MKI67",
    "C16ORF61": "CMC2",
    "C20ORF46": "TMEM74B",
    "C9ORF30": "MSANTD3",
    "GUS": "GUSB",
    "RPLPO": "RPLP0",
    "TRFC": "TFRC",
    "TSBP": "TSBP1"
}
invalid_genes = set()
def valid_gene(gene):
    gene = gene_name_mapper.get(gene, gene)
    if gene_lookup(gene) != 'None':
        return True
    invalid_genes.add(gene)
    return False

In [48]:
conditions = {}
invalid_conditions = set()
def get_condition_id(condition):
	if condition in conditions: return conditions[condition]
	else:
		res = requests.get("https://www.ebi.ac.uk/ols4/api/v2/entities?search=%s&ontologyId=doid"%condition)
		results = res.json()
		if results['numElements'] > 0:
			for e in results["elements"]:
				if type(e["label"]) == str and e["label"].lower() == condition.lower() or ("synonym" in e and condition.lower() in [d.lower() for d in e["synonym"]]):
					conditions[condition] = {
						"label": e["label"],
						"onto_id": e["curie"],
						"source": e["ontologyPreferredPrefix"]
					}
					break
			else:
				invalid_conditions.add(condition)
				return None
			

		


In [51]:
patents = {
	"WIPO": "https://patentscope.wipo.int/search/en/detail.jsf?docId=%s",
	"Google Patents": "https://patents.google.com/patent/%s/en"
}

In [69]:
gmt = []
biomarkers = []

with open('data/020524_biomarkers') as o:
    for line in o:
        group, c, pubmed, patent_id, patent_source, biomarker_temp, best_biomarker_role, *genes = line.split("\t")
        conditon = get_condition_id(c)
        biomarker = {
            "biomarker_id": "GeneSet_%s"%len(biomarkers),
            "best_biomarker_role": best_biomarker_role.replace("_biomarker", ""),
            "conditon": {
                "conditon_id": conditon["onto_id"],
                "recommended_name": {
                    "condition_id": conditon["onto_id"],
                    "name": conditon["label"],
                    "resource": "Disease Ontology",
                    "url": "http://disease-ontology.org/term/%s"%conditon["onto_id"],
                }
            },
        }
        evidence_source = []
        evidence_source.append({
            "evidence_id": pubmed,
            "database": "PubMed",
            "url": "https://pubmed.ncbi.nlm.nih.gov/%s"%pubmed
        })
        if patent_id:
            evidence_source.append({
                "evidence_id": patent_id,
                "database": patent_source,
                "url": patents[patent_source]%patent_id
            })
        biomarker["evidence_source"] = evidence_source
        biomarker_component = []
        for gene in genes:
            gene = gene.strip().upper()
            if valid_gene(gene):
                gene = gene_name_mapper.get(gene, gene)
                b = {
                    "biomarker": biomarker_temp.replace("{gene}", gene),
                    "assessed_biomarker_entity": {
                        "recommended_name": gene
                    },
                    "assessed_biomarker_entity_id": gene_lookup(gene),
                    "assessed_entity_type": ncbi_gene[ncbi_gene.GeneID == int(gene_lookup(gene))]["type_of_gene"].to_list()[0] + " gene",
                }
                biomarker_component.append(biomarker_component)
        
        biomarker["biomarker_component"] = biomarker_component
        biomarkers.append(biomarker)
                


In [71]:
len(biomarkers)

9

In [74]:
schema = requests.get("https://raw.githubusercontent.com/biomarker-ontology/biomarker-partnership/main/schema/v0.3.1/biomarker_schema.json").json()
sample = requests.get("https://raw.githubusercontent.com/biomarker-ontology/biomarker-partnership/main/supplementary_files/sample_data_model_structures/v0.3/sample_biomarker.json").json()

In [75]:
validate(instance=biomarkers, schema=schema)

ValidationError: 'prognostic' is not of type 'array'

Failed validating 'type' in schema['items']['properties']['best_biomarker_role']:
    {'description': 'Categories of BEST biomarker.',
     'items': {'properties': {'role': {'description': 'Category of BEST '
                                                      'biomarker.',
                                       'examples': ['risk', 'diagnostic'],
                                       'pattern': '^(risk|diagnostic|prognostic|monitoring|predictive|response|safety)$',
                                       'type': 'string'}},
               'required': ['role'],
               'type': 'object'},
     'type': 'array'}

On instance[0]['best_biomarker_role']:
    'prognostic'

In [ ]:
with open('out/020524_biomarker')